In [1]:
!pip install -r /content/drive/MyDrive/Thesis_Codes/requirements.txt


ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/content/drive/MyDrive/Thesis_Codes/requirements.txt'


In [2]:
import numpy as np
import functools
from scipy.sparse import csr_matrix
import pickle
import pymongo
from bson import ObjectId, decode_all

filename = "project_author_files_dict.pkl"


In [3]:
def row_normalize(M):
    U = M
    for i in range(U.shape[0]):
        rowsum = U[i].sum()
        if rowsum > 0:
            U[i] = U[i] / rowsum
    return U

def custom_sort(a, b):
    if a[0] > b[0]:
        return -1
    elif a[0] == b[0] and a[1] < b[1]:
        return -1
    return 1

In [4]:
with open(filename, "rb") as f:
    project_author_file_dict = pickle.load(f)
print("Dictionary loaded from pickle file:")

dict_items = list(project_author_file_dict.items())

def create_df_matrix(project_author_file_dict):
    unique_developers = set()
    unique_files = set()
    for project, authors_files in project_author_file_dict.items():
        for author, files in authors_files.items():
            unique_developers.add(author)
            unique_files.update(files)

    developer_index = {developer: idx for idx, developer in enumerate(unique_developers)}
    file_index = {file: idx for idx, file in enumerate(unique_files)}
    rows = []
    cols = []
    for project, authors_files in project_author_file_dict.items():
        for author, files in authors_files.items():
            dev_idx = developer_index[author]
            for file in files:
                file_idx = file_index[file]
                rows.append(dev_idx)
                cols.append(file_idx)
    df_matrix = csr_matrix((np.ones(len(rows)), (rows, cols)), shape=(len(unique_developers), len(unique_files)))
    return df_matrix, developer_index, file_index


Dictionary loaded from pickle file:


In [5]:
# HRank ASym Function
def HRank_ASym(A, B, UAP, UP, MPUPC, alpha=0.15):
    # Restart vector
    E_restart1 = np.full(len(A), 1/len(A))
    E_restart2 = np.full(len(B), 1/len(B))

    # Initial rank
    Vis_Prob1 = E_restart1.reshape(1, -1)    # Reshape for consistency
    Vis_Prob2 = E_restart2.reshape(1, -1)

    # Calculate MCP
    MCP = np.dot(UAP, np.dot(UP, MPUPC))

    # HRank iterations
    prev_iter1 = np.full(len(A), 1)
    prev_iter2 = np.full(len(B), 1)

    cn = 0
    while True:
        Vis_Prob1_copy = Vis_Prob1.copy()
        Vis_Prob2_copy = Vis_Prob2.copy()

        # Updated equation incorporating the provided formula
        Vis_Prob1 = alpha * np.dot(Vis_Prob1_copy, MCP) + (1 - alpha) * E_restart1
        Vis_Prob2 = alpha * np.dot(Vis_Prob2_copy, MCP.T) + (1 - alpha) * E_restart2

        # Check for convergence
        diff = max(np.max(np.absolute(np.nan_to_num(prev_iter1 - Vis_Prob1))), np.max(np.absolute(np.nan_to_num(prev_iter2 - Vis_Prob2))))
        if diff < 0.0001:
            break

        prev_iter1 = Vis_Prob1
        prev_iter2 = Vis_Prob2
        cn += 1

    # Sort and extract ranks
    rank1 = sorted(zip(Vis_Prob1[0], A), key=functools.cmp_to_key(custom_sort))
    rank2 = sorted(zip(Vis_Prob2[0], B), key=functools.cmp_to_key(custom_sort))

    return [x[1] for x in rank1], [x[1] for x in rank2]

In [6]:
import warnings
warnings.filterwarnings("ignore")

# Function to read and decode a BSON file
def read_bson_file(file_path):
    with open(file_path, 'rb') as f:
        data = f.read()
        documents = decode_all(data)
    return documents

# Load BSON data from files
final_identity_data = read_bson_file('final_identity.bson')
people_data = read_bson_file('people.bson')


# Convert to dictionaries for quick lookup
final_identity_dict = {entry['_id']: entry for entry in final_identity_data}
people_dict = {entry['_id']: entry for entry in people_data}

In [7]:
# Function to find people IDs from final_identity_data
def get_people_ids(final_identity_dict, dev_id):
    entry = final_identity_dict.get(ObjectId(dev_id))
    if entry:
        return entry['people']
    return []

# Function to find name and email by people ID from people_data
def find_name_and_email(people_dict, people_id):
    person = people_dict.get(ObjectId(people_id))
    if person:
        return person.get('name'), person.get('email')
    return None, None

# Initialize the results list
results = []
displayed_names = set()

# Rank developers for each project
for mm in range(1, 6):
    first_20_elements = dict_items[mm-1:mm]
    project_name = first_20_elements[0][0]
    first_20_dict = dict(first_20_elements)

    # Call the function
    df_matrix, developer_index, file_index = create_df_matrix(first_20_dict)

    developers = list(developer_index.keys())
    files = list(file_index.keys())
    reviewers = developers

    M_p1 = np.random.rand(len(reviewers), len(developers))
    M_p2 = np.random.rand(len(developers), len(reviewers))

    # Call the HRank_ASym function with the extracted information
    ranked_developers, _ = HRank_ASym(developers, reviewers, M_p1, M_p2, M_p2)

    # Extract top 20 ranked developer IDs
    top_20_ids = [developer for developer in ranked_developers[:20]]

    project_results = []
    project_displayed_names = set()

    # Match IDs in final_identity.bson and then in people.bson
    for dev_id in top_20_ids:
        people_ids = get_people_ids(final_identity_dict, dev_id)
        for people_id in people_ids:
            name, email = find_name_and_email(people_dict, people_id)
            if name and email:
                name_lower = name.lower()
                if name_lower not in project_displayed_names:
                    project_results.append({
                        'developer_id': dev_id,
                        'name': name,
                        'email': email
                    })
                    project_displayed_names.add(name_lower)

    # Print the top 20 developers' details for each project
    print("\n---------------------------------------------------------")
    print(f"-------------Project {project_name}----------------------")
    print("Top 20 Ranked Developers")
    for result in project_results:
        print(f"Developer ID: {result['developer_id']}, Name: {result['name']}, Email: {result['email']}")


---------------------------------------------------------
-------------Project commons-math----------------------
Top 20 Ranked Developers
Developer ID: 636529d5e9a99c9121bb254c, Name: dIon Gillard, Email: dion@multitask.com.au
Developer ID: 636529d5e9a99c9121bb254c, Name: dion, Email: dion@apache.org
Developer ID: 636529d5e9a99c9121bb254c, Name: dion gillard (JIRA), Email: commons-dev@jakarta.apache.org
Developer ID: 636529d5e9a99c9121bb254c, Name: dion.gillard@gmail.com, Email: dion.gillard@gmail.com
Developer ID: 636529d5e9a99c9121bb254c, Name: 'Dion Gillard', Email: dion.gillard@gmail.com
Developer ID: 636529d5e9a99c9121bb254c, Name: 'dion@multitask.com.au', Email: dion@multitask.com.au
Developer ID: 636529d5e9a99c9121bb254c, Name: dion.gillard, Email: dion.gillard@gmail.com
Developer ID: 636529d5e9a99c9121bb254c, Name: 'dIon Gillard ', Email: dion@multitask.com.au
Developer ID: 636529d5e9a99c9121bb254c, Name: dion_gillard, Email: dion_gillard@multitask.com.au
Developer ID: 636529